# 畳み込みニューラルネットワーク

## 畳み込み層

- 畳み込み層: CNN（Convolutional Neural Network）で新たに取り入れられる層の一つ。
- 特徴マップ(feature map): 畳み込み層における入出力データのこと
- 入力特徴マップ(input feature map): 畳み込み層における入力データのこと
- 出力特徴マップ(output feature map): 畳み込み層における出力データのこと

http://www.hpc.co.jp/AboutDeepLearning.html
![http://www.hpc.co.jp/images/DL_kaisetsu_09.png](http://www.hpc.co.jp/images/DL_kaisetsu_09.png)

### 全結合層の問題点

これまでのニューラルネットワークは隣接する層のすべてのニューロンとニューロンの間で結合がある（fully-connected; 全結合;）状態だった。  
我々はこれまでそれぞれの結合部をAffineレイヤ（行列の内積計算）で実装しReLUにて発火させた。これは何の考慮もせずに入力値をそのまま行列に変換してニューラルネットワークを構築したことになる。つまり今回のMNISTの手書き文字の場合、元々あった縦・横などの「データの形状情報」やRGB（ピクセルの色合いの順序情報;R要素・G要素・B要素の区別;）が失われてしまうことになる。  
__データをそのまま全結合層の入力（行列）として扱った場合3次元以上の情報は失われてしまう！__  
例えば、画像を行列として扱った場合に失われる情報としては以下が考えられる
- ピクセルのR,G,B,Aのそれぞれの濃度
- 縦・横のピクセル幅
- 上下左右で連続したピクセル間の値の推移

### 畳み込み演算

畳み込み演算は画像処理で言う「フィルター演算」にあたる。  
http://www.hpc.co.jp/AboutDeepLearning.html  
![http://www.hpc.co.jp/images/DL_kaisetsu_10.png](http://www.hpc.co.jp/images/DL_kaisetsu_10.png)

「フィルター」はカーネルとも呼ばれ、入力データに対してあるウインドウサイズの区画で積和演算を行う。全結合のニューラルネットワークでは行列が重みパラメータとなっていたが、CNNの場合はこのフィルターの乗算内容が「重み」に対応する。  
積和演算の結果は各ウィンドウの位置ごとに取得され、結果は新たな出力データとなる。  
その出力データの行列にスカラをスカラ加算（行列の要素全てに等しく加算）することでバイアスも実現する。  
バイアスはスカラなので、1 x 1の行列となる。  

### パディング

入力データの大外にダミーの枠を用意することで、積和演算結果のサイズを調整できる。  
畳み込み演算を繰り返すと段々出力サイズが小さくなるので、最終的な出力サイズが1になることを防ぐことができる。  
畳み込み演算回数（隠れ層の数）によっては不要の場合がある。

### ストライド

フィルターの適用する位置の間隔のこと。積和演算が一回終わった時点でウィンドウがスライドする幅。  
パディングとは逆に、ストライドは大きくすると積和演算結果の出力サイズは小さくなる。